In [1]:
import torch
import torchtext
import pandas as pd

# df = pd.read_csv('BindingDB_All_202311.tsv', sep = '\t', on_bad_lines='skip', low_memory = False)
# df[:100]

In [2]:
# data = df[["Ligand SMILES", "Ki (nM)", "IC50 (nM)", "BindingDB Target Chain Sequence"]]
# data

In [3]:
# ki, ic = 0, 0
# data_comp = data.isna()
# for i in range(1, len(data_comp)):
#     if not data_comp.loc[i,"Ki (nM)"]:
#         ki+=1
#     if not data_comp.loc[i,"IC50 (nM)"]:
#         ic+=1
# print(ki, ic)

In [4]:
# more ic50 data, so train,validate,test on it
# index = data[pd.isna(data[ "IC50 (nM)"])].index
# data.drop(index, inplace=True)
# data = data[["Ligand SMILES", "IC50 (nM)", "BindingDB Target Chain Sequence"]]
# data

In [5]:
# import re

# def ic_processing(ic50):
#     ic50_processed = float(re.sub('[^\d]','', ic50))
#     return ic50_processed

# def aa_processing(aa):
#     aa_processed = re.sub('[^A-Za-z]', '', aa).upper()
#     return aa_processed
    
# data['BindingDB Target Chain Sequence'] = data['BindingDB Target Chain Sequence'].apply(lambda x: aa_processing(str(x)))
# data['IC50 (nM)'] = data['IC50 (nM)'].apply(lambda x: ic_processing(str(x)))

In [6]:
# data

In [7]:
# data_bd = data[data['IC50 (nM)'] <= 500]

In [8]:
data_drop = pd.read_csv('data_drop.csv')
data_drop

,Ligand SMILES,IC50 (nM),BindingDB Target Chain Sequence
0,CC(C)[C@H](NC(C)=O)C(=O)N[C@@H](Cc1ccccc1)[C@@...,85,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...
1,CC(C)C[C@]1(N=CC([C@@H](Cc2ccccc2)C(N)=O)C1=O)...,10,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
2,CC(C)[C@H](NC(=O)c1ccc2ccccc2n1)C(=O)N[C@@H](C...,10,PQFSLWKRPVVTAYIEGQPVEVLLDTGADDSIVAGIELGNNYSPKI...
3,Cc1cc(NCN2C(=O)c3ccccc3C2=O)c(=O)[nH]c1C,218,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGK...
4,CC(C)[C@H](NC(=O)[C@@H](C[C@H](O)[C@H](Cc1cccc...,11,PQVTLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
...,...,...,...
4737,Cc1c(Br)cc(NC(=O)CS(O)(=O)=O)cc1Br,62,MAEVGSKSVLFVCLGNICRSPIAEAVFRKLVTDEKVSDNWRIDSAA...
4738,Cc1c(Br)cc(NC(=O)CS(O)(=O)=O)cc1Br,62,MAEVGSKSVLFVCLGNICRSPIAEAVFRKLVTDENVSDNWRIDSAA...
4739,NCCNS(=O)(=O)c1ccc(cc1)-c1ccc(CSc2nc3CCCc3c(=O...,83,MIRLGAPQTLVLLTLLVAAVLRCQGQDVQEAGSCVQDGQRYNDKDV...
4740,COC(=O)C[C@H]1[C@@]2(C)[C@H](C[C@@H]3CC(C(C)=C...,240,MAAQQRDCGGAAQLAGPAAEADPLGRFTCPVCLEVYEKPVQVPCGH...


In [9]:
# data_bd.to_csv('data_bd.csv')

In [10]:
# data_drop = data_bd.drop_duplicates(subset = ["BindingDB Target Chain Sequence"], keep = 'first')
# data_drop

In [11]:
# data_drop.to_csv('data_drop.csv')

In [13]:
# train, valid, test dataset split, iterable dataset으로 변환
from torch.utils.data.dataset import random_split

_train = []
for data_list in data_drop.values:
  _train.append(data_list)

num_train = int(len(_train) * 0.90)
num_test = int(len(_train)*0.05)
_train, _valid, _test = random_split(_train, [num_train, len(_train) - num_train - num_test, num_test])


In [14]:
from torchtext.vocab import vocab
from torchtext.data.utils import get_tokenizer
from deepchem.feat.smiles_tokenizer import BasicSmilesTokenizer
from torchtext.vocab import build_vocab_from_iterator

train_iter = iter(_train)
valid_iter = iter(_valid)
test_iter = iter(_test)

tokenize_smiles = BasicSmilesTokenizer()

def yield_tokens(file_iter, type = 0):
     if type == 0:
        for line in file_iter:
            smi_token = list(tokenize_smiles.tokenize(str(line[0])))
            smi_token.append('<eos>')
            smi_token.insert(0, '<sos>')
            yield smi_token
     else:
        for line in file_iter:
            aa_token = list(line[2])
            aa_token.append('<eos>')
            aa_token.insert(0, '<sos>')
            yield aa_token
            
smiles_voc = build_vocab_from_iterator(yield_tokens(train_iter, 0), specials=["<unk>", "<pad>", "<sos>", "<eos>"])
train_iter = iter(_train)
aa_voc = build_vocab_from_iterator(yield_tokens(train_iter, 1), specials=["<unk>", "<pad>", "<sos>", "<eos>"])
smiles_voc.set_default_index(smiles_voc['<unk>'])
aa_voc.set_default_index(aa_voc['<unk>'])

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
C:\Users\matth\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Skipped loading some PyTorch models, missing a dependency. No module named 'torch_geometric'
No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [15]:
print(len(smiles_voc))
print(len(aa_voc))

77
29


In [16]:
def be(a_list):
    a_list.append('<eos>')
    a_list.insert(0, '<sos>')
    return a_list

smiles_pipeline = lambda x: smiles_voc(be(tokenize_smiles.tokenize(str(x))))
aa_pipeline = lambda x: aa_voc(be(list(x)))

In [17]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torch import nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [32]:
def custom_collate_fn(batch):
    aa_list, smiles_list= [], []
    for [_smi, _, _aa] in batch:
         processed_smi = torch.tensor(smiles_pipeline(_smi), dtype=torch.int64)
         smiles_list.append(processed_smi)
         processed_aa = torch.tensor(aa_pipeline(_aa), dtype=torch.int64)
         aa_list.append(processed_aa)
    smiles_list = pad_sequence(smiles_list, padding_value = 1)
    aa_list = pad_sequence(aa_list, padding_value = 1)
    
    return aa_list.to(device), smiles_list.to(device)

In [33]:
train_iter = iter(_train)
valid_iter = iter(_valid)
test_iter = iter(_test)

train_dataloader = DataLoader(list(train_iter), batch_size=16,
                              shuffle=True, collate_fn=custom_collate_fn)

valid_dataloader = DataLoader(list(valid_iter), batch_size=16,
                              shuffle=True, collate_fn=custom_collate_fn)

test_dataloader = DataLoader(list(test_iter), batch_size=16,
                              shuffle=True, collate_fn=custom_collate_fn)

In [34]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_dim, emb_dim)

        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)

        self.dropout = nn.Dropout(dropout)

    def forward(self, aa):

        #aa = [aa len, batch size]

        embedded = self.dropout(self.embedding(aa))

        #embedded = [aa len, batch size, emb dim]

        outputs, (hidden, cell) = self.rnn(embedded)

        #outputs = [aa len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]

        #outputs are always from the top hidden layer

        return hidden, cell

In [35]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)

        self.fc_out = nn.Linear(hid_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):

        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]

        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]

        input = input.unsqueeze(0)
        #input = [1, batch size]

        embedded = self.dropout(self.embedding(input))

        #embedded = [1, batch size, emb dim]

        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))

        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]

        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]

        prediction = self.fc_out(output.squeeze(0))

        #prediction = [batch size, output dim]

        return prediction, hidden, cell

In [74]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"

    def forward(self, aa, smi, teacher_forcing_ratio = 0.5):

        #aa = [aa len, batch size]
        #smi = [smi len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        if smi is None:
            smi = torch.zeros((150, aa.shape[1])).fill_(2).long().to(aa.device)
            assert teacher_forcing_ratio == 0, "Must be zero during inference"
            
        batch_size = smi.shape[1]
        smi_len = smi.shape[0]
        smi_vocab_size = self.decoder.output_dim

        #tensor to store decoder outputs
        outputs = torch.zeros(smi_len, batch_size, smi_vocab_size).to(self.device)

        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(aa)

        #first input to the decoder is the <sos> tokens
        input = smi[0,:]

        for t in range(1, smi_len): # <sos> token 이미 사용했으므로 1부터 시작

            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)

            #place predictions in a tensor holding predictions for each token
            outputs[t] = output

            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio

            #get the highest predicted token from our predictions
            top1 = output.argmax(1)

            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = smi[t] if teacher_force else top1

        return outputs

In [37]:
INPUT_DIM = len(aa_voc)
OUTPUT_DIM = len(smiles_voc)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [38]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(29, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(77, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=77, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [39]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 7,423,053 trainable parameters


In [40]:
optimizer = torch.optim.Adam(model.parameters())

In [41]:
SMI_PAD_IDX = smiles_voc(['<pad>'])
criterion = nn.CrossEntropyLoss(ignore_index = SMI_PAD_IDX[0])

In [42]:
def train(model, iterator, optimizer, criterion, clip):

    model.train()
    log_interval = 10
    epoch_loss = 0

    for i, batch in enumerate(iterator):
    
        aa = batch[0]
        smi = batch[1]
        batch_loss = 0

        optimizer.zero_grad()

        output = model(aa, smi)

        #smi = [smi len, batch size]
        #output = [smi len, batch size, output dim]

        output_dim = output.shape[-1]

        output = output[1:].view(-1, output_dim)
        smi = smi[1:].view(-1)

        #smi = [(smi len - 1) * batch size]
        #output = [(smi len - 1) * batch size, output dim]

        loss = criterion(output, smi)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()
        batch_loss += loss.item()
        
        if i % log_interval == 0 and i > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| loss {:8.2f}'.format(epoch+1, i, len(iterator),
                                           batch_loss))

    return epoch_loss / len(iterator)

In [43]:
def evaluate(model, iterator, criterion):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for i, batch in enumerate(iterator):

            aa = batch[0]
            smi = batch[1]

            output = model(aa, smi, 0) #turn off teacher forcing

            #smi = [smi len, batch size]
            #output = [smi len, batch size, output dim]

            output_dim = output.shape[-1]

            output = output[1:].view(-1, output_dim)
            smi = smi[1:].view(-1)

            #smi = [(smi len - 1) * batch size]
            #output = [(smi len - 1) * batch size, output dim]

            loss = criterion(output, smi)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [44]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [45]:
import time
import random
import math

N_EPOCHS = 6
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_dataloader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_dataloader, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'cello-model.pt')

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

| epoch   1 |    10/  267 batches | loss     2.53
| epoch   1 |    20/  267 batches | loss     2.48
| epoch   1 |    30/  267 batches | loss     2.53
| epoch   1 |    40/  267 batches | loss     2.48
| epoch   1 |    50/  267 batches | loss     2.38
| epoch   1 |    60/  267 batches | loss     2.36
| epoch   1 |    70/  267 batches | loss     2.20
| epoch   1 |    80/  267 batches | loss     2.38
| epoch   1 |    90/  267 batches | loss     2.13
| epoch   1 |   100/  267 batches | loss     2.05
| epoch   1 |   110/  267 batches | loss     2.20
| epoch   1 |   120/  267 batches | loss     2.24
| epoch   1 |   130/  267 batches | loss     2.08
| epoch   1 |   140/  267 batches | loss     2.16
| epoch   1 |   150/  267 batches | loss     1.93
| epoch   1 |   160/  267 batches | loss     2.22
| epoch   1 |   170/  267 batches | loss     2.21
| epoch   1 |   180/  267 batches | loss     1.95
| epoch   1 |   190/  267 batches | loss     1.94
| epoch   1 |   200/  267 batches | loss     1.91


| epoch   6 |   250/  267 batches | loss     1.33
| epoch   6 |   260/  267 batches | loss     1.96
Epoch: 06 | Time: 20m 12s
	Train Loss: 1.664 | Train PPL:   5.280
	 Val. Loss: 3.660 |  Val. PPL:  38.860


In [75]:
INPUT_DIM = len(aa_voc)
OUTPUT_DIM = len(smiles_voc)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

SMI_PAD_IDX = smiles_voc(['<pad>'])
criterion = nn.CrossEntropyLoss(ignore_index = SMI_PAD_IDX[0])

model = Seq2Seq(enc, dec, device).to(device)
model.load_state_dict(torch.load('cello-model.pt'))
model.eval()

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(29, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(77, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=77, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [64]:
EPOCHS = 1

for epoch in range(EPOCHS):
    test_loss = evaluate(model, test_dataloader, criterion)
    end_time = time.time()
    print(f'\tTest Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f}')

	Test Loss: 2.733 | Test PPL:  15.372


In [51]:
_train[0]

array(['Cc1ncsc1-c1ccc(CNC(=O)[C@@H]2C[C@@H](O)CN2C(=O)[C@@H](NC(=O)COCCOCCOCCN2CCN(Cc3ccc(cc3)-c3cc4c(NCc5ccccc5Cl)ncnc4[nH]3)CC2)C(C)(C)C)cc1',
       210,
       'MRPSGTAGAALLALLAALPASRALEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYVQRNYDLSFLKTIQEVAGYVLIALNTVERIPLENLQIIRGNMYYENSYALAVLSNYDANKTGLKELPMRNLQEILHGAVRFSNNPALCNVESIQWRDIVSSDFLSNMSMDFQNHLGSCQKCDPSCPNGSCWGAGEENCQKLTKIICAQQCSGRCRGKSPSDCCHNQCAAGCTGPRESDCLVCRKFRDEATCKDTCPPLMLYNPTTYQMDVNPEGKYSFGATCVKKCPRNYVVTDHGSCVRACGADSYEMEEDGVRKCKKCEGPCRKVCNGIGIGEFKDSLSINATNIKHFKNCTSISGDLHILPVAFRGDSFTHTPPLDPQELDILKTVKEITGFLLIQAWPENRTDLHAFENLEIIRGRTKQHGQFSLAVVSLNITSLGLRSLKEISDGDVIISGNKNLCYANTINWKKLFGTSGQKTKIISNRGENSCKATGQVCHALCSPEGCWGPEPRDCVSCRNVSRGRECVDKCNLLEGEPREFVENSECIQCHPECLPQAMNITCTGRGPDNCIQCAHYIDGPHCVKTCPAGVMGENNTLVWKYADAGHVCHLCHPNCTYGCTGPGLEGCPTNGPKIPSIATGMVGALLLLLVVALGIGLFMRRRHIVRKRTLRRLLQERELVEPLTPSGEAPNQALLRILKETEFKKIKVLGSGAFGTVYKGLWIPEGEKVKIPVAIKLRETSPKANKEILDEAYVMASVDNPHVCRLLGICLTSTVQLITQLMPFGCLLDYVREHKDNIGSQYLLNWCVQIAKGMNYLEDRRLVHRD

In [85]:
def generate_inhibitor(aa_seq):
    tokenized = tokenize_smiles.tokenize(aa_seq) #tokenize sentence
    tokenized = be(list(tokenized))
    numericalized = aa_voc.lookup_indices(tokenized) #convert tokens into indexes
    print(numericalized)
    tensor = torch.LongTensor(numericalized).unsqueeze(1).to(device) #convert to tensor and add batch dimension
    smiles_tensor_probs = model(tensor, None, 0).squeeze(1) #pass through model to get translation probabilities
    print(smiles_tensor_probs)
    smiles_tensor = torch.argmax(smiles_tensor_probs, 1) #get translation from highest probabilities
    smiles = smiles_voc.lookup_tokens(list(smiles_tensor))[1:] #we ignore the first token, just like we do in the training loop
    return ''.join(smiles)

In [87]:
candidate = _train[8][2]
candidate_inhibitor = _train[8][0]

print(f'Amino acid sequence: {candidate}')
print(f'Inhibitor: {candidate_inhibitor}')

Amino acid sequence: MRDTRFPCLCGIAASTLLFATTPAIADEAPADRLKALVDAAVQPVMKANDIPGLAVAISLKGEPHYFSYGLASKEDGRRVTPETLFEIGSVSKTFTVTLAGYALAQDKMRLDDRASQHWPALQGSRFDGISLLDLATYTAGGLPLQFPDSVQKDQAQIRDYYRQWQPTYAPGSQRLYSNPSIGLFGYLAARSLGQPFERLMEQQLFPALGLEQTHLDVPEAALAQYAQGYGKDDRPLRVGPGPLDAEGYGVKTSAADLLRFVDANLHPERLDRPWAQALDATHRGYYKVGDMTQGLGWEAYDWPISLKRLQAGNSTPMALQPHRIARLPAPQALEGQRLLNKTGSTNGFGAYVAFVPGRDLGLVILANRNYPNAERVKIAYAILSGLEQQAKVPLKR
Inhibitor: OS(=O)(=O)ON1[C@H]2CN([C@H](CC2)C(=O)NC2CCCNC2)C1=O


In [88]:
generate_inhibitor(candidate)

[2, 16, 11, 22, 22, 14, 5, 16, 11, 14, 11, 11, 17, 14, 11, 14, 5, 11, 16, 5, 5, 11, 16, 14, 5, 5, 16, 5, 11, 5, 16, 14, 5, 11, 16, 11, 5, 14, 11, 11, 5, 5, 17, 11, 5, 14, 16, 5, 11, 16, 16, 11, 11, 11, 11, 11, 5, 16, 17, 11, 11, 11, 14, 5, 17, 5, 11, 11, 14, 11, 11, 17, 5, 17, 16, 16, 11, 14, 17, 17, 11, 17, 14, 14, 5, 11, 3]
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-4.9521, -4.4609, -5.3924,  ..., -4.5506, -4.2073, -3.9494],
        [-6.0279, -6.2610, -6.3815,  ..., -5.5365, -5.9922, -5.3801],
        ...,
        [-6.0936, -6.1561, -6.0791,  ..., -5.9719, -6.1138, -5.6587],
        [-6.0937, -6.1561, -6.0793,  ..., -5.9720, -6.1140, -5.6587],
        [-6.0938, -6.1561, -6.0794,  ..., -5.9721, -6.1141, -5.6587]],
       grad_fn=<SqueezeBackward1>)


'CC(C)CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC'

In [90]:
generate_inhibitor('EDQIDSPR')

[2, 14, 5, 11, 3]
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-4.8542, -4.4335, -5.3823,  ..., -4.5392, -4.0643, -3.9426],
        [-6.0422, -6.0604, -6.3918,  ..., -5.5688, -5.8116, -5.3007],
        ...,
        [-5.7102, -6.2847, -6.0608,  ..., -5.8109, -4.4862, -5.1523],
        [-6.2263, -6.3437, -6.3282,  ..., -5.3459, -5.0593, -6.3578],
        [-6.9668, -6.5130, -6.6230,  ..., -6.2314, -7.0949, -6.5225]],
       grad_fn=<SqueezeBackward1>)


'CC(C)CC)C(=O)N[C@@H](CCCNC(N)=N)C(=O)N[C@@H](CCCNC(N)=N)NC(=O)[C@H](CCCNC(N)=N)NC(=O)[C@H](CCCNC(N)=N)NC(=O)[C@H](CCCNC(N)=N)NC(=O)[C@H](CCCNC(N)=N)NC(=O)[C@H](CCCNC(N)=N)NC(=O)[C@H]('

In [93]:
candidate = _test[67][2]
candidate_inhibitor = _test[67][0]

print(f'Amino acid sequence: {candidate}')
print(f'Inhibitor: {candidate_inhibitor}')

Amino acid sequence: MGSKRGISSRHHSLSSYEIMFAALFAILVVLCAGLIAVSCLTIKESQRGAALGQSHEARATFKITSGVTYNPNLQDKLSVDFKVLAFDLQQMIDEIFLSSNLKNEYKNSRVLQFENGSIIVVFDLFFAQWVSDENVKEELIQGLEANKSSQLVTFHIDLNSVDILDKLTTTSHLATPGNVSIECLPGSSPCTDALTCIKADLFCDGEVNCPDGSDEDNKMCATVCDGRFLLTGSSGSFQATHYPKPSETSVVCQWIIRVNQGLSIKLSFDDFNTYYTDILDIYEGVGSSKILRASIWETNPGTIRIFSNQVTATFLIESDESDYVGFNATYTAFNSSELNNYEKINCNFEDGFCFWVQDLNDDNEWERIQGSTFSPFTGPNFDHTFGNASGFYISTPTGPGGRQERVGLLSLPLDPTLEPACLSFWYHMYGENVHKLSINISNDQNMEKTVFQKEGNYGDNWNYGQVTLNETVKFKVAFNAFKNKILSDIALDDISLTYGICNGSLYPEPTLVPTPPPELPTDCGGPFELWEPNTTFSSTNFPNSYPNLAFCVWILNAQKGKNIQLHFQEFDLENINDVVEIRDGEEADSLLLAVYTGPGPVKDVFSTTNRMTVLLITNDVLARGGFKANFTTGYHLGIPEPCKADHFQCKNGECVPLVNLCDGHLHCEDGSDEADCVRFFNGTTNNNGLVRFRIQSIWHTACAENWTTQISNDVCQLLGLGSGNSSKPIFPTDGGPFVKLNTAPDGHLILTPSQQCLQDSLIRLQCNHKSCGKKLAAQDITPKIVGGSNAKEGAWPWVVGLYYGGRLLCGASLVSSDWLVSAAHCVYGRNLEPSKWTAILGLHMKSNLTSPQTVPRLIDEIVINPHYNRRRKDNDIAMMHLEFKVNYTDYIQPICLPEENQVFPPGRNCSIAGWGTVVYQGTTANILQEADVPLLSNERCQQQMPEYNITENMICAGYEEGGIDSCQGDSGGPLMCQE

In [94]:
generate_inhibitor(candidate)

[2, 5, 14, 5, 5, 5, 5, 5, 14, 16, 16, 14, 22, 14, 5, 22, 14, 5, 5, 16, 14, 5, 17, 11, 17, 5, 16, 16, 14, 14, 16, 5, 5, 17, 17, 17, 5, 16, 17, 5, 14, 14, 16, 16, 16, 5, 17, 14, 17, 5, 5, 16, 14, 17, 5, 14, 5, 11, 17, 5, 14, 22, 11, 5, 5, 11, 22, 22, 14, 16, 22, 17, 22, 11, 5, 17, 22, 22, 16, 5, 5, 5, 16, 11, 11, 5, 5, 22, 14, 14, 17, 5, 14, 5, 16, 16, 17, 14, 14, 5, 5, 14, 5, 14, 17, 11, 14, 14, 16, 5, 17, 16, 14, 5, 5, 16, 17, 16, 17, 5, 5, 17, 17, 14, 17, 22, 17, 16, 16, 22, 16, 17, 17, 14, 5, 16, 5, 11, 16, 11, 17, 16, 16, 17, 5, 16, 14, 5, 11, 11, 5, 11, 11, 11, 22, 5, 16, 17, 5, 14, 17, 14, 5, 17, 17, 16, 17, 17, 17, 17, 16, 16, 17, 16, 17, 14, 5, 14, 14, 5, 14, 22, 17, 5, 11, 11, 11, 11, 11, 11, 11, 22, 11, 16, 11, 17, 16, 5, 5, 17, 16, 11, 17, 5, 11, 17, 16, 22, 14, 17, 17, 14, 16, 16, 17, 14, 17, 14, 5, 11, 11, 16, 5, 17, 14, 17, 16, 17, 16, 14, 11, 11, 22, 16, 22, 17, 22, 11, 17, 22, 22, 5, 22, 16, 16, 17, 17, 17, 17, 16, 14, 5, 14, 22, 17, 14, 5, 17, 22, 5, 17, 5, 5, 11, 14, 1

'CC(C)CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC'